In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
from collections import Counter
from typing import NamedTuple, List, Dict, Tuple
import random
from datetime import datetime
import time
import math

# config

In [4]:
class Config(NamedTuple):
    num_units: int = 512
    num_layers: int = 6
    embedding_size = 512
    batch_size: int = 128
    max_length: int = 50
    dropout_in_rate: float = 0.1
    dropout_out_rate: float = 0.2
    learning_rate: float = 0.001
    grad_clip: float = 5.0
    is_layer_norm: bool = False
    checkpoint_dir = './checkpoints/'
    data_path: str = './data/'
    log_dir = './logs/'
    
    def to_log_dir(self) -> str:
        return self.log_dir + 'layers={}/units={}/lr={}'.format(self.num_layers, self.num_units, self.learning_rate)
    
    def to_ckpt_path(self) -> str:
        layernorm = 'T' if self.is_layer_norm else 'F'
        return self.checkpoint_dir + 'l{}_u{}_lr{}_clip_ln{}_model.ckpt'.format(self.num_layers, self.num_units, self.learning_rate, layernorm)

In [5]:
units = [512, 1024]
layers = [4]
lrs = [0.001]
configs = []
for l in layers:
    for u in units:
        for lr in lrs:
            configs.append(Config(num_layers=l, num_units=u, learning_rate=lr))

# data

In [6]:
class PTBDataSource:
    
    def __init__(self, config: Config):
        self.config = config
        train_path = os.path.join(self.config.data_path, 'ptb.train.txt')
        test_path = os.path.join(self.config.data_path, 'ptb.test.txt')
        valid_path = os.path.join(self.config.data_path, 'ptb.valid.txt')
        
        self._word_to_id = self._create_tokenizer(train_path)
        self._id_to_word = {v: k for k, v in self._word_to_id.items()}
        
        self.train = self._create_data(train_path)
        self.test = self._create_data(test_path)
        self.valid = self._create_data(valid_path)
        
    def shuffle(self):
        random.shuffle(self.train)
        
    def feed_dict_list(self, model):
        num_batch = len(self.train) // self.config.batch_size
        batch_list = []
        inputs, inputs_length, target_ids = self._make_feed_list(self.train)
        
        # batch_sizeに分ける
        for i in range(num_batch):
            index_from = i * self.config.batch_size
            index_to = (i + 1) * self.config.batch_size
            batch_range = range(index_from, index_to)
            fd = {
                model.inputs: inputs[batch_range],
                model.inputs_length: inputs_length[batch_range],
                model.target_ids: target_ids[batch_range]
            }
            batch_list.append(fd)
        return batch_list
    
    def feed_test_list(self, model):
        test_data = random.sample(self.test, 256)
        inputs, inputs_length, target_ids = self._make_feed_list(test_data)
        fd = {
            model.inputs: inputs,
            model.inputs_length: inputs_length,
            model.target_ids: target_ids
        }
        return fd

    def _make_feed_list(self, data):
        inputs = []
        inputs_length = []
        target_ids = []
        for (i, sentence) in enumerate(data):
            for j in range(len(sentence)-1):
                inputs_words = sentence[:j+1][-self.config.max_length:] 
                inputs.append(inputs_words + [0] * (self.config.max_length - len(inputs_words)))
                inputs_length.append(len(inputs_words))
                target_ids.append(sentence[j+1])
        inputs = np.array(inputs)
        inputs_length = np.array(inputs_length)
        target_ids = np.array(target_ids)
        
        return inputs, inputs_length, target_ids

    def _read_all_words(self, path) -> List[str]:
        with open(path, 'r') as f:
            return f.read().replace('\n', '<eos>').split()
        
    def _read_sentences(self, path) -> List[List[str]]:
        with open(path, 'r') as f:
            sentences = f.read().split('\n')
            return [sentence.split() for sentence in sentences]

    def _create_tokenizer(self, path: str):
        data = self._read_all_words(path)
        counter = Counter(data)
        sorted_counter = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
        words, _ = list(zip(*sorted_counter))
        word_to_id = dict(zip(words, range(1, len(words)+1)))
        return word_to_id
        
    def _get_id_from_word(self, word: str) -> int:
        return self._word_to_id.get(word, self.unk_id)
    
    def _sentence_to_id_list(self, sentence: List[str]) -> List[int]:
        return [self._get_id_from_word(word) for word in sentence]
    
    def _get_word_from_id(self, word_id: int) -> str:
        return self._id_to_word.get(word_id, self.unk_str)
    
    def _create_data(self, path: str):
        return [self._sentence_to_id_list(sentence) for sentence in self._read_sentences(path)]
    
    @property
    def vocab_size(self) -> int:
        return len(self._word_to_id)
    
    @property
    def pad_id(self) -> int:
        return 0
    
    @property
    def unk_id(self) -> int:
        return self._word_to_id.get('<unk>', self.pad_id)
    
    @property
    def eos_id(self) -> int:
        return self._word_to_id.get('<eos>', self.pad_id)

# model

In [9]:
class RNN:
    
    def __init__(self, config: Config, vocab_size):
        self.config = config
        self.vocab_size = vocab_size
        self._create_placeholder()
        self._create_model()
        self.loss = self._create_loss()
        self.accuracy = self._create_acc()
    
    def _create_placeholder(self):
        self.is_training = tf.placeholder(shape=(), dtype=tf.bool, name='is_training')
        self.inputs = tf.placeholder(shape=[None, self.config.max_length], dtype=tf.int32, name='inputs')
        self.inputs_length = tf.placeholder(shape=[None], dtype=tf.int32, name='inputs_length')
        self.target_ids = tf.placeholder(shape=[None], dtype=tf.int32, name='target_ids')
    
    def _create_model(self):
        self.global_step = tf.train.get_or_create_global_step()
        embedded_inputs = self._embedding(self.inputs)
        _, encoder_state = self._encode(embedded_inputs)
        # encoder_state = tf.layers.dense(encoder_state, num_units, activation=tf.nn.relu, name='hidden_layer')
        self.outputs_logits = tf.layers.dense(encoder_state, self.vocab_size, name='outputs_layer')
        self.predicted_id = tf.to_int32(tf.argmax(self.outputs_logits, axis=-1))
        
    def _create_loss(self):
        is_target = tf.to_float(tf.not_equal(self.target_ids, 0))
        target_ids_one_hot = tf.one_hot(self.target_ids, self.vocab_size)
        target_ids_smoothed = self._label_smoothing(target_ids_one_hot)
        cross_ent = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.outputs_logits, labels=target_ids_smoothed)
        return tf.reduce_sum(cross_ent * is_target) / tf.reduce_sum(is_target)
        
    def _create_acc(self):
        return tf.reduce_mean(tf.to_float(tf.equal(self.target_ids, self.predicted_id)))
    
    def _embedding(self, inputs):
        lookup_table = tf.get_variable('lookup_table', shape=[self.vocab_size, self.config.embedding_size], dtype=tf.float32)
        embedded_inputs = tf.nn.embedding_lookup(lookup_table, inputs)
        return embedded_inputs
    
    def _encode(self, embedded_inputs):
        outputs, final_state = self._bidirectional_cell(
            embedded_inputs,
            self.config.num_layers,
            self.config.num_units,
            self.config.dropout_in_rate,
            self.config.dropout_out_rate
        )
        return outputs, final_state
    
    def _bidirectional_cell(self, inputs, num_layers, num_units, dropout_in_rate, dropout_out_rate):
        cell_fw = self._gru(num_layers, num_units, dropout_in_rate, dropout_out_rate, name='cell_fw')
        cell_bw = self._gru(num_layers, num_units, dropout_in_rate, dropout_out_rate, name='cell_bw')
        (fw_outputs, bw_outputs), (fw_state, bw_state) = tf.nn.bidirectional_dynamic_rnn(
            cell_fw=cell_fw,
            cell_bw=cell_bw,
            inputs=inputs,
            sequence_length=self.inputs_length,
            dtype=tf.float32,
            scope='bidirectional_cells')
        outputs = tf.concat([fw_outputs, bw_outputs], axis=-1)
        final_state = tf.reduce_sum([fw_state, bw_state], axis=0)
        final_state = tf.concat(tf.unstack(final_state, axis=0), axis=-1)
        return outputs, final_state
    
    def _gru(self, num_layers: int, num_units: int, dropout_in_rate: float, dropout_out_rate: float, name: str):
        cells = []
        for l in range(num_layers):
            cell = tf.nn.rnn_cell.GRUCell(num_units, tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(), name=name)
            if l == 0:
                cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=1-dropout_in_rate)
            if l == num_layers-1:
                cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=1-dropout_out_rate)
            cells.append(cell)
        return tf.nn.rnn_cell.MultiRNNCell(cells)
    
    def _label_smoothing(self, inputs, epsilon: float=0.1):
        feature_dim = inputs.get_shape().as_list()[-1]
        return (1-epsilon) * inputs + (epsilon / feature_dim)

In [10]:
num_epoch = 200

In [11]:
def start():
    with tf.Graph().as_default():
        now = datetime.now()
        logdir = now.strftime("%Y%m%d-%H%M%S") + "/"

        datasource = PTBDataSource(config)

        rnn = RNN(config, datasource.vocab_size)
        optimizer = tf.train.AdamOptimizer(config.learning_rate)
        train_vars = tf.trainable_variables()
        gradients = tf.gradients(rnn.loss, train_vars)
        clipped_gradients, _ = tf.clip_by_global_norm(gradients, config.grad_clip)
        train_op = optimizer.apply_gradients(zip(clipped_gradients, train_vars), global_step=rnn.global_step)
        with tf.name_scope('training'):
            s_loss = tf.summary.scalar('loss', rnn.loss)
            s_acc = tf.summary.scalar('accuracy', rnn.accuracy)
        with tf.name_scope('test'):
            test_s_acc = tf.summary.scalar('accuracy', rnn.accuracy)

        with tf.Session() as sess:
            saver = tf.train.Saver()
            writer = tf.summary.FileWriter(config.to_log_dir() + '/' + logdir, sess.graph)
            sess.run(tf.global_variables_initializer())
            for i in range(num_epoch):
                start = time.time()
                datasource.shuffle()
                batch_list = datasource.feed_dict_list(rnn)
                losses = []
                accuracies = []
                for (j, fd) in enumerate(batch_list):
                    loss, acc, _, smr_loss, smr_acc, step = sess.run([rnn.loss, rnn.accuracy, train_op, s_loss, s_acc, rnn.global_step], feed_dict=fd)
                    losses.append(loss)
                    accuracies.append(acc)
                    writer.add_summary(smr_loss, step)
                    writer.add_summary(smr_acc, step)
                    if j % 100 == 0:
                        #print('loss: {:.3f}, acc: {:.3f}'.format(loss, acc))
                        inference(sess, rnn, datasource, writer, test_s_acc, step)
                elapsed = time.time() - start
                print('epoch {}/{} finished, {} step, elapsed {} sec. average loss: {:.3f}, average accuracy: {:.3f}'.format(i+1, num_epoch, step, elapsed, np.average(losses), np.average(accuracies)))
                # loss が nan なら 飛ばす
                if math.isnan(np.average(losses)):
                    print('loss is nan')
                    break
                saver.save(sess, config.to_ckpt_path(), global_step=step)

In [12]:
def inference(sess, model, datasource, writer, s_acc, step):
    with tf.name_scope('inference'):
        test_list = datasource.feed_test_list(model)
        acc, smr_acc = sess.run([model.accuracy, s_acc], feed_dict=test_list)
        writer.add_summary(smr_acc, step)

In [ ]:
for config in configs:
    start()

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead
epoch 1/200 finished, 327 step, elapsed 63.05414271354675 sec. average loss: 7.291, average accuracy: 0.066
epoch 2/200 finished, 655 step, elapsed 62.784444093704224 sec. average loss: 6.763, average accuracy: 0.122
epoch 3/200 finished, 983 step, elapsed 64.81405925750732 sec. average loss: 6.470, average accuracy: 0.148
epoch 4/200 finished, 1311 step, elapsed 64.68730759620667 sec. average loss: 6.336, average accuracy: 0.161
epoch 5/200 finished, 1639 step, elapsed 66.68140697479248 sec. average loss: 6.196, average accuracy: 0.175
epoch 6/200 finished, 1967 step, elapsed 65.44995546340942 sec. average loss: 6.112, average accuracy: 0.184
epoch 7/200 finished, 2295 step, elapsed 65.29462242126465 sec. average loss: 6.018, average accuracy: 0.190
epoch 8/200 finished, 2623 step, elapsed 64.58694672584534 sec. average loss: 5.995, average 